# Reddit Longevity Evidence Agent - Google Colab

This notebook runs the entire pipeline in Google Colab (free GPU tier).

**Before running:**
1. Get Reddit API credentials from https://www.reddit.com/prefs/apps
2. Fill in the credentials in Cell 2
3. Run all cells (Runtime → Run all)
4. Download results when complete

**Expected runtime:** ~60-90 minutes

## 1. Setup Environment

In [ ]:
# Clone repository
!git clone https://github.com/yourname/longevity-reddit-agent.git
%cd longevity-reddit-agent

# Install dependencies
!pip install -q praw pandas requests python-dotenv pyarrow ollama

## 2. Configure Credentials

**IMPORTANT:** Replace these with your actual Reddit API credentials!

In [ ]:
import os

# Reddit API credentials
os.environ["REDDIT_CLIENT_ID"] = "your_client_id_here"
os.environ["REDDIT_CLIENT_SECRET"] = "your_client_secret_here"
os.environ["REDDIT_USERNAME"] = "your_reddit_username"
os.environ["REDDIT_PASSWORD"] = "your_reddit_password"
os.environ["REDDIT_USER_AGENT"] = "longevity-agent by u/your_username"

# Persist to Google Drive (optional)
# from google.colab import drive
# drive.mount('/content/drive')
# os.environ["DATA_DIR"] = "/content/drive/MyDrive/longevity_agent/data"

print("✓ Credentials configured")

## 3. Install Ollama (Local LLM)

In [ ]:
# Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# Pull model (this will take 5-10 minutes)
!ollama pull llama3.2:3b

print("✓ Ollama installed and model downloaded")

## 4. Collect Reddit Posts

Fetches last 365 days of r/longevity posts (~5-10 minutes)

In [ ]:
!python src/01_collect.py

## 5. Extract Claims

Extracts longevity claims using LLM (~30-45 minutes)

In [ ]:
!python src/02_extract_claims.py

## 6. Check Evidence

Verifies claims against PubMed (~45-60 minutes)

In [ ]:
!python src/03_evidence_check.py

## 7. Preview Results

In [ ]:
import pandas as pd
import glob

# Load latest results
files = glob.glob("data/processed/claims_evidence_*.csv")
latest_file = max(files)
df = pd.read_csv(latest_file)

print(f"✓ Loaded {len(df)} claims from {latest_file}")
print(f"\nEvidence distribution:")
print(df["evidence_level"].value_counts())

print(f"\nTop 10 topics:")
print(df["topic"].value_counts().head(10))

# Show sample
print(f"\nSample claims:")
df.head(10)[["claim", "topic", "evidence_level", "post_score"]]

## 8. Download Results

In [ ]:
from google.colab import files

# Download CSV
files.download(latest_file)

print("✓ Downloaded! Check your Downloads folder.")

## 9. Optional: Generate Report

In [ ]:
# Generate markdown report
report = f"""# r/longevity Evidence Report

Generated: {pd.Timestamp.now().strftime("%Y-%m-%d %H:%M")}

## Summary
- Total claims: {len(df)}
- Unique topics: {df['topic'].nunique()}
- Strong evidence: {len(df[df['evidence_level'] == 'strong_support'])}

## Evidence Distribution
{df['evidence_level'].value_counts().to_markdown()}

## Top 10 Most Upvoted Claims
"""

for idx, row in df.nlargest(10, 'post_score').iterrows():
    report += f"\n### {row['claim']}\n"
    report += f"- Topic: {row['topic']}\n"
    report += f"- Evidence: {row['evidence_level']}\n"
    report += f"- Reddit Score: {row['post_score']}\n\n"

# Save report
with open("longevity_report.md", "w") as f:
    f.write(report)

files.download("longevity_report.md")
print("✓ Report generated and downloaded!")